In [1]:
source_dir = '../tests/test_data/test_ancor_bank/test_waveforms_multi_station'
target_dir = '../tests/test_data/test_ancor_bank/test_save_output'
from obsplus.bank import WaveBank
from obspy.core import Stream, Trace
import numpy as np
import sys
import os
sys.path.append('/home/kevin/PycharmProjects/ancor')
starttime_stamp = 1481761092.0 + 3600 * 24


In [2]:
class WavebankWrapper(AnxcorDatabase):

    def __init__(self, directory):
        self.bank = WaveBank(directory)
        import warnings
        warnings.filterwarnings("ignore")

    def get_waveforms(self, **kwargs):
        stream =  self.bank.get_waveforms(**kwargs)
        traces = []
        for trace in stream:
            data = trace.data[:-1]
            header = {'delta':np.floor(trace.stats.delta*1000)/1000.0,
                      'station': trace.stats.station,
                      'starttime':trace.stats.starttime,
                      'channel': trace.stats.channel,
                      'network': trace.stats.network}
            traces.append(Trace(data,header=header))
        return Stream(traces=traces)

    def get_stations(self):
        df = self.bank.get_uptime_df()

        def create_seed(row):
            network = row['network']
            station = row['station']
            return network + '.' + station

        df['seed'] = df.apply(lambda row: create_seed(row), axis=1)
        unique_stations = df['seed'].unique().tolist()
        return unique_stations


In [3]:
from anxcor.core import Anxcor
from anxcor.xarray_routines import XArrayBandpass, XArrayRemoveMeanTrend, XArrayTemporalNorm, XArrayWhiten

In [4]:
anxcor = Anxcor(3600)
times = anxcor.get_starttimes(starttime_stamp, starttime_stamp + 4 * 3600, 0.5)
bank = WavebankWrapper(source_dir)
anxcor.add_dataset(bank, 'nodals')

anxcor.set_task_kwargs('crosscorrelate',{'max_tau_shift':10.0})

anxcor.add_process(XArrayBandpass(lower_frequency=0.001))
anxcor.add_process(XArrayRemoveMeanTrend())
anxcor.add_process(XArrayTemporalNorm())
anxcor.add_process(XArrayWhiten(lower_frequency=0.001))

In [5]:
from dask.distributed import Client

In [6]:
client = Client(n_workers=1,threads_per_worker=4)


In [7]:
client

Client Scheduler: tcp://127.0.0.1:37409 Dashboard: http://127.0.0.1:35737/status,Cluster Workers: 1 Cores: 4 Memory: 25.22 GB


In [8]:
result=anxcor.process(times,dask_client=client)

In [9]:
import traceback  # doctest: +SKIP
tb = result.traceback()  # doctest: +SKIP
traceback.format_tb(tb)

['  File "/home/kevin/PycharmProjects/ancor/anxcor/abstractions.py", line 166, in _execute\n    persisted_metadata = self.__metadata_to_persist(*args, **kwargs)\n',
 '  File "/home/kevin/PycharmProjects/ancor/anxcor/abstractions.py", line 222, in __metadata_to_persist\n    return self._metadata_to_persist(*param,**kwargs)\n',
 '  File "/home/kevin/PycharmProjects/ancor/anxcor/xarray_routines.py", line 258, in _metadata_to_persist\n    attrs = {\'delta\'    : xarray_1.attrs[\'delta\'],\n']

In [15]:
from IPython.display import display
display(r)

In [9]:
result = result.result()

True
True
False
True
